In [ ]:
!pip install -q datasets 
!pip install -q parsivar
!pip install -q hazm
!pip install -q transformers 

## Imports

In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict, Counter
from tqdm.notebook import tqdm_notebook
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset, DatasetDict
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from sklearn.metrics import accuracy_score
from transformers import AdamW
from parsivar import Normalizer
import hazm


## Load data

In [4]:
ls


baseline.ipynb             output_data/
data/                      stop-words.txt
data_df.csv                Torob/
data_df.pkl                torob_contest.ipynb
data_trob20.csv            torob_contest_pairwise_dataset_v2.ipynb
data_trob_without_sql.csv  torob-data-challenge-2023_datafiles_v1.7z
eda.ipynb                  torob_v3_model_learning.ipynb
graph_torob_contest.ipynb  train_data_complete.csv
LaBSE/                     train_data_complete.csv.zip
my_database.db             Untitled.ipynb


In [6]:
# !unzip /content/drive/MyDrive/train_data_complete.csv.zip

In [5]:
data_df = pd.read_csv("./train_data_complete.csv", encoding="utf-8")
data_df.head()


,Unnamed: 0,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,relevance
0,0,لوستر سقفی برنز,6462477,لوستر سقفی چشمه نور 5 شعله کد C2542-B برنز,2.0,کاپشن، بارانی و پالتو زنانه,3.0,0.012945,22.0,1.0,1.440678,821000.0,[],unpopular,3.0
1,1,لوستر سقفی برنز,7385791,لوستر سقفی چشمه نور 3 شعله کد C2542/3B برنز,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003236,22.0,0.0,1.440678,792000.0,['۳'],unpopular,0.0
2,2,لوستر سقفی برنز,3775536,لوستر سقفی مدرن مدل نیلوفر سه تایی- سفید,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.0,1.440678,400000.0,[],unpopular,0.0
3,3,لوستر سقفی برنز,1663988,لوستر سقفی کریستالی دایره سایز 25 مدل کارمانیا,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.0,1.440678,756000.0,[],unpopular,0.0
4,4,لوستر سقفی برنز,6946932,لوستر سقفی کریستالی دایره سایز 25 دوبلکس مدل آداس,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.0,1.440678,756000.0,[],unpopular,0.0


## Load Model

In [7]:
c_model = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(c_model)

## Preprocess Data

In [8]:
data_df

,Unnamed: 0,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,relevance
0,0,لوستر سقفی برنز,6462477,لوستر سقفی چشمه نور 5 شعله کد C2542-B برنز,2.0,کاپشن، بارانی و پالتو زنانه,3.0,0.012945,22.0,1.00000,1.440678,821000.0,[],unpopular,3.0
1,1,لوستر سقفی برنز,7385791,لوستر سقفی چشمه نور 3 شعله کد C2542/3B برنز,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003236,22.0,0.00000,1.440678,792000.0,['۳'],unpopular,0.0
2,2,لوستر سقفی برنز,3775536,لوستر سقفی مدرن مدل نیلوفر سه تایی- سفید,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,400000.0,[],unpopular,0.0
3,3,لوستر سقفی برنز,1663988,لوستر سقفی کریستالی دایره سایز 25 مدل کارمانیا,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,756000.0,[],unpopular,0.0
4,4,لوستر سقفی برنز,6946932,لوستر سقفی کریستالی دایره سایز 25 دوبلکس مدل آداس,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,756000.0,[],unpopular,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571356,1571356,کاپشن زنانه ترک,4987530,کاپشن زنانه ترک کد HBIN 80,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,1602840.0,[],unpopular,0.0
1571357,1571357,کاپشن زنانه ترک,5323788,خرید کاپشن زنانه ترک برند Koton رنگ مشکی کد ty...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,1000000.0,"['Dik', 'Yaka', 'Mont', 'Şişme', 'Fermuarlı']",unpopular,0.0
1571358,1571358,کاپشن زنانه ترک,6993184,خرید نقدی کاپشن زنانه ترک برند TAMER COLLECTİ...,1.0,کاپشن، بارانی و پالتو زنانه,1.0,0.014184,2.0,0.63093,1.000000,1920000.0,"['Vizon', 'Mont', 'Kapüşonlu', 'Beden', 'Ultra...",unpopular,4.0
1571359,1571359,کاپشن زنانه ترک,7938070,Kahverengi Oversize Çıt Çıt Kapamalı Kapitone ...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,840000.0,"['ty195352872', 'رنگ', 'قهوه', 'کد', 'ترک', 'پ...",unpopular,0.0


In [9]:
data_df.drop(columns=["Unnamed: 0"], inplace=True)


In [11]:
data_df = data_df.sample(frac=1, random_state=40).reset_index()
data_df


,index,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,relevance
0,1090233,اتو صورت اولتراسونیک,5113124,دستگاه اتوی پوست ( درما اف )التراسونیک اتو صور...,2.807355,کاپشن، بارانی و پالتو زنانه,6.0,0.027559,15.0,0.701839,5.101695,299000.0,"['کره', '3', 'اُف', '(اتوی', '–', 'پاکسازی', '...",popular,4.0
1,314452,چکمه دخترانه,1027590,چکمه دخترانه کد daft468,1.584963,کاپشن، بارانی و پالتو زنانه,2.0,0.005300,22.0,0.366726,1.000000,1064000.0,[],unpopular,1.0
2,385867,تب لت,5521963,تبلت 11 اینچی گلکسی S8 سامسونگ با ظرفیت 128 گی...,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.005747,7.0,0.000000,18.864407,21990000.0,"['8', 'Galaxy', 'گیگ', 'RAM', 'X700', '8GB', '...",unpopular,0.0
3,225785,ساعت دوخترانه,8891734,ساعت مچی آیس مدل 001065 | گالری ساعت آماتیست,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.007874,7.0,0.000000,2.084746,1835000.0,"['واچ,', 'Small', 'کد', '۰۰۱۰۶۵', 'واچ', 'زنانه']",unpopular,0.0
4,935905,زه کمان,426465,بازوی کمان ریکرو فیویکس مدل Xenia Wood,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.011494,9.0,0.000000,1.550000,4875000.0,[],unpopular,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571356,1176866,تاچ ال سی‌دی a 12,476213,ال سی دی شرکتی گوشی گلکسی آ10 سامسونگ Samsung...,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.040000,6.0,0.000000,27.555556,395000.0,"['Pack', 'M105', 'and', 'M10/', 'CHANGE', 'فری...",popular,0.0
1571357,783274,ریمل کالیستا بنفش,3038679,ریمل حجم دهنده ضد آب کالیستا,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.011494,2.0,0.000000,5.203390,55000.0,[],unpopular,0.0
1571358,1496618,دور فرمان پژو,5364121,دور فرمان حلقه ای چرم مصنوعی | روکش فرمان خودرو,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.003717,3.0,0.000000,1.283333,35000.0,[],unpopular,0.0
1571359,1543543,وان پلاس نورد,6185645,"گلس وان پلاس نورد CE ایی 2 لایت , OnePlus Nord...",0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.023810,12.0,0.000000,2.900000,50000.0,[],unpopular,0.0


## Normalizer

In [12]:
class MyNormalizer:
    def __init__(self):
        self.parsivar_normalizer = Normalizer(
            statistical_space_correction=True,
            half_space_char=" ",
            pinglish_conversion_needed=True,
        )
        self.hazm_normalizer = hazm.Normalizer(
            remove_extra_spaces=True,
            persian_numbers=True,
            persian_style=True,
            punctuation_spacing=False,
            remove_diacritics=True,
            affix_spacing=False,
            token_based=True,
        )

    def normilize(self, txt):
        return self.hazm_normalizer.normalize(
            self.parsivar_normalizer.normalize(
                txt.replace("\n", " ").replace("\u200c", " ").lower().strip()
            )
        )


In [13]:
normalizer = MyNormalizer()


In [14]:
normalizer.normilize("34.0")


'۳۴ ٫۰'

In [15]:
def preprocess(record):
    query = normalizer.normilize(record["query"])
    p_des = normalizer.normilize(record["p_des"])
    popularity = str(record["popularity"])
    new_words = " ".join(record["new words"])

    encoded_text = tokenizer(
        query,
        popularity + " " + p_des + " " + new_words,
        truncation=True,
        max_length=512,
    )

    label = record["relevance"] * 10

    return {
        "input_ids": encoded_text["input_ids"],
        "attention_mask": encoded_text["attention_mask"],
        "token_type_ids": encoded_text["token_type_ids"],
        "label": label,
    }


In [17]:
train_df = data_df[:1500000]
test_df = data_df[-71361:]


In [18]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [20]:
train_dataset, test_dataset


(Dataset({
     features: ['index', 'query', 'p_id', 'p_des', 'label', 'category_name', 'clicks', 'ctr', 'impressions', 'log_normalized_candidate_score', 'mean_max_shops', 'min_price', 'new words', 'popularity', 'relevance'],
     num_rows: 1500000
 }),
 Dataset({
     features: ['index', 'query', 'p_id', 'p_des', 'label', 'category_name', 'clicks', 'ctr', 'impressions', 'log_normalized_candidate_score', 'mean_max_shops', 'min_price', 'new words', 'popularity', 'relevance'],
     num_rows: 71361
 }))

In [ ]:
train_dataset = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(preprocess, remove_columns=test_dataset.column_names)


Parameter 'function'=<function preprocess at 0x7f6f20ab8ef0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1500000 [00:00<?, ?ex/s]

In [ ]:
train_dataset = train_dataset.shuffle()
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")


In [ ]:
train_dataset.save_to_disk("./train_dataset")
test_dataset.save_to_disk("./test_dataset")


# Model

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, c_model):
        super(BaseModel, self).__init__()
        self.bert = AutoModel.from_pretrained(c_model)
        self.fc = nn.Linear(self.bert.config.hidden_size, 256)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256, 1)
        

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        out = self.fc(output[1])
        return self.fc2(self.relu(out))


## Validation, Train, Test

In [ ]:
def validation(model, val_loader, loss_fn, test=True, device="cuda"):
    m = nn.Sigmoid()
    val_loss = 0.0
    model.eval()
    reals = []
    preds_list = []
    for batch in val_loader:
        input_ids = batch["input_ids"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"]
        with torch.no_grad():
            preds = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )
        loss = loss_fn(preds.squeeze(1), labels.to(device).to(torch.float32))
        val_loss += loss.item()
    val_loss = val_loss / len(val_loader)
    return val_loss


In [ ]:
def train(
    model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda"
):
    m = nn.Sigmoid()
    model.to(device)
    for epoch in range(epochs):
        training_loss = 0.0
        val_loss = 0.0
        model.train()
        t_labels = []
        t_preds = []
        for batch in tqdm_notebook(train_loader):
            input_ids = batch["input_ids"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"]
            t_labels += labels.numpy().tolist()
            optimizer.zero_grad()
            preds = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )
            loss = loss_fn(preds.squeeze(1), labels.to(device).to(torch.float32))
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
        training_loss = training_loss / len(train_loader)
        print("Epoch: {},training loss: {:.4f}  ".format(epoch, training_loss))
    test_loss = validation(model, test_loader, loss_fn, device=device)
    print("test loss: {:.2f} ".format(test_loss))


In [ ]:
def collate_batch(batch):
    input_ids = []
    attention_mask = []
    token_type_ids = []
    labels = []
    for b in batch:
        input_ids.append(b["input_ids"])
        token_type_ids.append(b["token_type_ids"])
        attention_mask.append(b["attention_mask"])
        labels.append(b["label"])
    input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids, batch_first=True, padding_value=0.0
    )
    attention_mask = torch.nn.utils.rnn.pad_sequence(
        attention_mask, batch_first=True, padding_value=0.0
    )
    token_type_ids = torch.nn.utils.rnn.pad_sequence(
        token_type_ids, batch_first=True, padding_value=0.0
    )
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "token_type_ids": token_type_ids,
        "label": torch.Tensor(labels).to(torch.long),
    }


## Model Initialization

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaseModel(c_model)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.MSELoss()

## 1st cut

In [ ]:
train_dataset = Dataset.from_disk("./train_dataset")
test_dataset = Dataset.from_disk("./test_dataset")
train_data = train_dataset.select(range(0, 500000))

In [ ]:
# train_dataset = train_dataset.shuffle()
train_data.set_format(type='torch')
test_dataset.set_format(type='torch')

In [ ]:
train_data, test_dataset

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=16, collate_fn=collate_batch, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, collate_fn=collate_batch, shuffle=True
)


In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")


In [ ]:
torch.save(
    model.state_dict(), "/content/drive/MyDrive/" + "torobv3_bert_epoch-1_1st" + ".w"
)

## 2nd cut

In [ ]:
model.load_state_dict(
    torch.load("./content/drive/MyDrive/" + "torobv3_bert_epoch-1_1st" + ".w"),
    strict=True,
)

In [ ]:
train_data = train_dataset.select(range(500000, 1000000))
train_data.set_format(type="torch")

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=16, collate_fn=collate_batch, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, collate_fn=collate_batch, shuffle=True
)


In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")


  0%|          | 0/12500 [00:00<?, ?it/s]

Epoch: 0,training loss: 5.7007  
test loss: 5.12 


In [ ]:
torch.save(
    model.state_dict(), "/content/drive/MyDrive/" + "torobv3_bert_epoch-1_2nd" + ".w"
)


## 3rd cut

In [ ]:
model.load_state_dict(
    torch.load("./content/drive/MyDrive/" + "torobv3_bert_epoch-1_2nd" + ".w"),
    strict=True,
)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data , batch_size=16, collate_fn=collate_batch ,shuffle=True)

  0%|          | 0/100000 [00:00<?, ?ex/s]

In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")

  0%|          | 0/6250 [00:00<?, ?it/s]

Epoch: 0,training loss: 1.0281  
test loss: 0.99 


In [ ]:
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/" + "torobv3_bert_epoch-1_3rd" + ".w",
)

## 4th cut

In [ ]:
train_df = data_df[750000:1000000]
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
train_dataset = train_dataset.shuffle()
train_dataset.set_format(type="torch")
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, collate_fn=collate_batch, shuffle=True
)


  0%|          | 0/100000 [00:00<?, ?ex/s]

In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")


  0%|          | 0/6250 [00:00<?, ?it/s]

In [ ]:
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/" + "torobv3_bert_epoch-1_4rd_250k" + ".w",
)


## 5th 250K

In [ ]:
train_df = data_df[1000000:1250000]
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
train_dataset = train_dataset.shuffle()
train_dataset.set_format(type="torch")
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, collate_fn=collate_batch, shuffle=True
)


In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")


In [ ]:
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/" + "torobv3_bert_epoch-1_5th_250k" + ".w",
)


## 6th 250K

In [ ]:
train_df = data_df[1000000:1250000]
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
train_dataset = train_dataset.shuffle()
train_dataset.set_format(type="torch")
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=16, collate_fn=collate_batch, shuffle=True
)


In [ ]:
train(model, optimizer, loss_fn, train_loader, test_loader, epochs=1, device="cuda")


In [ ]:
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/" + "torobv3_bert_epoch-1_6th_250k" + ".w",
)
